1.	What is a neural network? What are the general steps required to build a neural network? 

Neural networks are computing systems that are inspired by the neural networks in brains. They learn by example and can classify and make predictions.  

What are the steps? 

Clean the data

Select the architecture

train the neural network

tweak the performance

test

deploy model


2.	Generally, how do you check the performance of a neural network? Why? 


There are a bunch of methods to check the performance of regression

MSE

NMSE

RMSE

R Squared

For classification

TP rate

FP Rate

F Measure

Acurracy

Precision

Recall

Mean squared error is most common for regression,

Why do we want to check the performance? Because having an accurate model is the only reason for making a model. It is pointless to have a model with terrible performance. If you check the performance you can tweak a ton of different options to try to make it perform better. 

3.	Create a neural network using keras to predict the outcome of either of these datasets: 
Cardiac Arrhythmia: https://archive.ics.uci.edu/ml/datasets/Arrhythmia 
Abalone age: https://archive.ics.uci.edu/ml/datasets/Abalone


In [178]:
import pandas as pd

column_list = ['Sex','Length','Diameter','Height','Whole weight','Shucked weight','Viscera weight','Shell weight','Rings']
abalone_df = pd.read_table("abalone.data",sep=",",header=None,names=column_list)



In [179]:
abalone_df = pd.get_dummies(abalone_df)
abalone_df.head()

,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings,Sex_F,Sex_I,Sex_M
0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15,0,0,1
1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7,0,0,1
2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9,1,0,0
3,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10,0,0,1
4,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7,0,1,0


In [180]:
abalone_df = abalone_df[abalone_df.Height != 0]

In [181]:
abalone_df['Rings'].value_counts().sort_index()

1       1
2       1
3      15
4      57
5     115
6     258
7     391
8     567
9     689
10    634
11    487
12    267
13    203
14    126
15    103
16     67
17     58
18     42
19     32
20     26
21     14
22      6
23      9
24      2
25      1
26      1
27      2
29      1
Name: Rings, dtype: int64

In [182]:
abalone_df = abalone_df[(abalone_df['Rings'] < 21) & (abalone_df['Rings']>3)]
abalone_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4122 entries, 0 to 4176
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Length          4122 non-null   float64
 1   Diameter        4122 non-null   float64
 2   Height          4122 non-null   float64
 3   Whole weight    4122 non-null   float64
 4   Shucked weight  4122 non-null   float64
 5   Viscera weight  4122 non-null   float64
 6   Shell weight    4122 non-null   float64
 7   Rings           4122 non-null   int64  
 8   Sex_F           4122 non-null   uint8  
 9   Sex_I           4122 non-null   uint8  
 10  Sex_M           4122 non-null   uint8  
dtypes: float64(7), int64(1), uint8(3)
memory usage: 301.9 KB


In [184]:
out_abalone=abalone_df.copy()
Q1 = out_abalone.quantile(0.25)
Q3 = out_abalone.quantile(0.75)
IQR = Q3 - Q1
print(IQR)
abalone_df = out_abalone[~((out_abalone < (Q1 - 2.5 * IQR)) |(out_abalone > (Q3 + 2.5 * IQR))).any(axis=1)]
abalone_df.shape

Length            0.16500
Diameter          0.13500
Height            0.05000
Whole weight      0.70350
Shucked weight    0.31575
Viscera weight    0.15800
Shell weight      0.19225
Rings             3.00000
Sex_F             1.00000
Sex_I             1.00000
Sex_M             1.00000
dtype: float64


(4055, 11)

In [185]:
from sklearn import preprocessing
X = abalone_df.drop(['Rings'], axis=1)
y = abalone_df['Rings']


In [186]:
from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 27)

In [187]:
sc = StandardScaler()
sc.fit_transform(X, y)

array([[-0.57816635, -0.43195836, -1.16192728, ..., -0.66948808,
        -0.69510779,  1.31974141],
       [-1.46684626, -1.45682005, -1.29490355, ..., -0.66948808,
        -0.69510779,  1.31974141],
       [ 0.05660501,  0.13171558, -0.09811714, ...,  1.49367858,
        -0.69510779, -0.7577242 ],
       ...,
       [ 0.64905828,  0.69538951,  1.76355062, ..., -0.66948808,
        -0.69510779,  1.31974141],
       [ 0.86064874,  0.79787568,  0.30081167, ...,  1.49367858,
        -0.69510779, -0.7577242 ],
       [ 1.58005628,  1.51527887,  1.49759808, ..., -0.66948808,
        -0.69510779,  1.31974141]])

In [189]:
# Import necessary modules
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.callbacks import EarlyStopping


# Specify the model
n_cols = X.shape[1]
model = Sequential()
model.add(Dense(300, activation='relu', input_shape = (n_cols,)))
model.add(Dense(300, activation='relu'))
model.add(Dense(300, activation='relu'))
model.add(Dense(300, activation='relu'))
model.add(Dense(1, activation='relu'))
model.summary()

early_stopping_monitor = EarlyStopping(patience = 3)



model.summary()
# Compile the model
model.compile(optimizer = 'adam', loss = 'mse', metrics = ['mean_squared_error'])


#model.fit(X_train, y_train,  validation_split = .3, epochs = 30, batch_size=100,verbose=1,callbacks = [early_stopping_monitor])
model.fit(X, y, validation_split = .3, epochs = 30, batch_size=100,verbose=1,callbacks = [early_stopping_monitor])

                                                                              
                                                                              
                                                                 
                                                     

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_65 (Dense)             (None, 300)               3300      
_________________________________________________________________
dense_66 (Dense)             (None, 300)               90300     
_________________________________________________________________
dense_67 (Dense)             (None, 300)               90300     
_________________________________________________________________
dense_68 (Dense)             (None, 300)               90300     
_________________________________________________________________
dense_69 (Dense)             (None, 1)                 301       
Total params: 274,501
Trainable params: 274,501
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_13"
_________________________________________________________________
Layer (type)            

In [190]:
NNMSE = 3.4380
NNRMSE = NNMSE**.5
NNRMSE

1.8541844568434933

In [191]:
abalone_df.head()

,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings,Sex_F,Sex_I,Sex_M
0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15,0,0,1
1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7,0,0,1
2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9,1,0,0
3,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10,0,0,1
4,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7,0,1,0


4.	Write another algorithm to predict the same result as the previous question using either KNN or logistic regression.




In [192]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression


In [193]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 27)

In [194]:
from sklearn.metrics import mean_squared_error 

LR = LinearRegression()

LR.fit(X_train, y_train)

y_pred = LR.predict(X_test)


mse = mean_squared_error(y_test, y_pred)

rmse = mse**.5
rmse

1.908865138210307

In [195]:
LogReg = LogisticRegression(max_iter = 7500, random_state = 27)
LogReg.fit(X_train, y_train)
LogRegy_pred = LogReg.predict(X_test)

MSE = mean_squared_error(y_test, LogRegy_pred)

RMSE = MSE**.5
RMSE

2.3526414812280136

5.	Create a neural network using pytorch to predict the same result as question 3. 


In [207]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = abalone_df.drop('Rings', axis=1).values
y = abalone_df['Rings'].values

# Split into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=27)

# #Standardize
sc= StandardScaler()
X_train=sc.fit_transform(PTX_train)
X_test=sc.fit_transform(PTX_test)

In [208]:
import torch
import torch.nn as nn
import torch.nn.functional as F #this has activation functions

# Creating tensors
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)

#based on error messages further down, I think these should be float, but it got 
#very angry at me when I tried to change them to FloatTensor so we'll just 
#cast to float later. 
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

print(X_train)

tensor([[-0.8600, -0.9299, -0.6180,  ..., -0.6738,  1.4274, -0.7476],
        [ 0.5688,  0.6483,  0.3110,  ..., -0.6738, -0.7006,  1.3377],
        [ 1.5774,  1.6156,  1.7709,  ..., -0.6738, -0.7006,  1.3377],
        ...,
        [-0.1036, -0.0135, -0.3525,  ..., -0.6738, -0.7006,  1.3377],
        [-0.9440, -0.7772, -0.8834,  ..., -0.6738,  1.4274, -0.7476],
        [-0.0195, -0.2680, -0.3525,  ..., -0.6738,  1.4274, -0.7476]])


In [209]:
class ANN_Model(nn.Module):
    def __init__(self, input_features=10, hidden1=20, hidden2=20, out_features =1):
        super().__init__()
        self.layer_1_connection = nn.Linear(input_features, hidden1)
        self.layer_2_connection = nn.Linear(hidden1, hidden2)
        self.out = nn.Linear(hidden2, out_features)
    
    def forward(self, x):
        #apply activation functions
        x = F.relu(self.layer_1_connection(x))
        x = F.relu(self.layer_2_connection(x))
        x = self.out(x)
        return x

In [210]:
torch.manual_seed(27)

#instantiate the model
model = ANN_Model()

In [211]:
# loss function
loss_function = nn.MSELoss()


In [212]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)


In [216]:
#run model through multiple epochs/iterations
final_loss = []
n_epochs = 200
for epoch in range(n_epochs):
    y_pred = model.forward(X_train)
    loss = loss_function(y_pred.float(), y_train.float())
    final_loss.append(loss)
    
    if epoch % 10 == 1:
        print(f'Epoch number: {epoch} with loss: {loss.item()}')
    
    optimizer.zero_grad() #zero the gradient before running backwards propagation
    loss.backward() #for backward propagation 
    optimizer.step() #performs one optimization step each epoch

Epoch number: 1 with loss: 7.67382287979126
Epoch number: 11 with loss: 7.673658847808838
Epoch number: 21 with loss: 7.673503875732422
Epoch number: 31 with loss: 7.673357009887695
Epoch number: 41 with loss: 7.673215389251709
Epoch number: 51 with loss: 7.673080921173096
Epoch number: 61 with loss: 7.672955513000488
Epoch number: 71 with loss: 7.672837734222412
Epoch number: 81 with loss: 7.672726154327393
Epoch number: 91 with loss: 7.6726226806640625
Epoch number: 101 with loss: 7.672525405883789
Epoch number: 111 with loss: 7.672430038452148
Epoch number: 121 with loss: 7.672338962554932
Epoch number: 131 with loss: 7.672253131866455
Epoch number: 141 with loss: 7.672173976898193
Epoch number: 151 with loss: 7.672099590301514
Epoch number: 161 with loss: 7.6720290184021
Epoch number: 171 with loss: 7.671961307525635
Epoch number: 181 with loss: 7.671895980834961
Epoch number: 191 with loss: 7.671833515167236


In [217]:
torchRMSE = 7.671833515167236**.5
torchRMSE 

2.769807487022742

In [174]:
"""
import torch

y_pred = []

with torch.no_grad():
    for i, data in enumerate(X_test):
        prediction = model(data)
        y_pred.append(prediction.argmax().item())

"""

'\nimport torch\n\ny_pred = []\n\nwith torch.no_grad():\n    for i, data in enumerate(X_test):\n        prediction = model(data)\n        y_pred.append(prediction.argmax().item())\n\n'

In [55]:
#from sklearn.metrics import classification_report

#print(classification_report(y_test, y_pred))

6.	Compare the performance of the neural networks to the other model you created. Which performed better? Why do you think that is?


My Keras model performed the best. My guess is that by design it is better than linear or logistic regression. I think if I had a more solid understanding of what was happening in the pytorch version I might be able to get that down and better.

In [218]:
print('Keras: ' + str(NNRMSE))
print('LogisticRegression: ' + str(RMSE))
print('LinearRegression: ' + str(rmse))
print('Pytorch: ' + str(torchRMSE))



Keras: 1.8541844568434933
LogisticRegression: 2.3526414812280136
LinearRegression: 1.908865138210307
Pytorch: 2.769807487022742
